In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

C:\Users\vasil\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_datasets():
    # reading the datasets
    train = pd.read_csv('../input/train.csv.gz', compression='infer')
    infer = pd.read_csv('../input/test.csv.gz', compression='infer')
    
    train_features = train.drop(['Id','SalePrice'], axis=1)
    train_labels = train['SalePrice']
    infer_features = infer.drop('Id', axis=1)
    
    from sklearn.model_selection import train_test_split
    X_train, X_eval, y_train, y_eval = train_test_split(train_features, train_labels, test_size=0.15, random_state=42)
    
    def get_bins(df):
        bins = {}
        numeric_columns = [
            'LotFrontage', 'LotArea', 'TotalBsmtSF', 'BsmtFinSF2', 'BsmtUnfSF', 'WoodDeckSF',
            'BsmtFinSF1', 'MasVnrArea', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea',
            'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
        
        for column in numeric_columns:
            tempdf, retbins = pd.cut(df[column], 10, labels=None, retbins=True, include_lowest=True)
            bins[column] = retbins

        return bins 
    
    bins = get_bins(X_train)
    
    return X_train, X_eval, y_train, y_eval, infer_features, bins

X_train, X_eval, y_train, y_eval, infer_features, bins = load_datasets()

In [3]:
from tensorflow.feature_column import *

all_columns = X_train.columns
numeric_columns = [
        'LotFrontage', 'LotArea', 'TotalBsmtSF', 'BsmtFinSF2', 'BsmtUnfSF', 'WoodDeckSF',
        'BsmtFinSF1', 'MasVnrArea', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea',
        'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
categorical_columns = set(all_columns) - set(numeric_columns)

BUCKETIZED_FEATURES = [bucketized_column(numeric_column(column), boundaries=bins[column].tolist()) for column in 
                      numeric_columns]

CATEGORICAL_FEATURES = [categorical_column_with_identity(key=column, num_buckets=150) for column in categorical_columns]

FEATURE_COLUMNS = []
for feature in BUCKETIZED_FEATURES:
    FEATURE_COLUMNS.append(feature)
    
for feature in CATEGORICAL_FEATURES:
    FEATURE_COLUMNS.append(indicator_column(feature))
    
for column in categorical_columns:
    X_train[column], junk = pd.factorize(X_train[column])
    X_eval[column], junk = pd.factorize(X_eval[column])
    infer_features[column], junk = pd.factorize(infer_features[column])

In [4]:
def train_input_fn(features, labels):
    return tf.estimator.inputs.pandas_input_fn(
                x = features,
                y = labels,
                num_epochs = 2,
                shuffle = True)

def eval_input_fn(features, labels):
    return tf.estimator.inputs.pandas_input_fn(
                x = features,
                y = labels,
                num_epochs = 1,
                shuffle = True)

def pred_input_fn(features):
    return tf.estimator.inputs.pandas_input_fn(
                x = features,
                num_epochs = 1,
                shuffle = False)

In [5]:
def model_fn(features, labels, mode, params):
    # mapping the feature columns as input layer
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    
    # building the NN based on the list passed in 'hidden_units'
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)
        net = tf.layers.dropout(net, rate=0.1, training=True)
        
    # compute the result
    output_layer = tf.layers.dense(net, units=1)
    
    # Reshape the output layer to a 1-dim Tensor to return predictions
    value = tf.squeeze(output_layer, 1)
    
    # where to terminate if mode == predict
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=value)
    
    # computing the loss function

    loss = tf.sqrt(tf.losses.mean_squared_error(tf.log1p(labels), tf.log1p(value)))
    
    # where to terminate if mode == train
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # where to terminate if mode == eval
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss)

In [6]:
OUTDIR = './DNN_CUSTOM_REGRESSOR'

file_writer = tf.summary.FileWriter(OUTDIR)

estimator = tf.estimator.Estimator(
    model_fn=model_fn, 
    model_dir=OUTDIR, 
    params={
        'feature_columns':FEATURE_COLUMNS,
        'hidden_units':[128,64]
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './DNN_CUSTOM_REGRESSOR', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C3183845F8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
def combine_sets(f_tr, f_ev, y_tr, y_ev):
    f = pd.concat([f_tr, f_ev])
    y = pd.concat([y_tr, y_ev])
    
    return f,y

def random_split(X,y):
    from sklearn.model_selection import train_test_split
    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.35)
    
    return X_train, X_eval, y_train, y_eval

In [13]:
X, y = combine_sets(X_train, X_eval, y_train, y_eval)

def train_and_evaluate(estimator, epochs=1000):
    for i in range(epochs):
        X_train, X_eval, y_train, y_eval = random_split(X,y)
        estimator.train(input_fn=train_input_fn(X_train,y_train.astype('float32')))
        estimator.evaluate(input_fn=eval_input_fn(X_eval,y_eval.astype('float32')))
        
train_and_evaluate(estimator)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6480
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6480 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:loss = 0.14244655, step = 6481
INFO:tensorflow:Saving checkpoints for 6495 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:Loss for final step: 0.1748959.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-12-18:05:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6495
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-12-18:05:49
INFO:tensorflow:Saving dict

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6585
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6585 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:loss = 0.16581172, step = 6586
INFO:tensorflow:Saving checkpoints for 6600 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:Loss for final step: 0.14809042.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-12-18:07:58
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-12-18:08:00
INFO:tensorflow:Saving dic

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6690
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6690 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:loss = 0.17293344, step = 6691
INFO:tensorflow:Saving checkpoints for 6705 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:Loss for final step: 0.15943559.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-12-18:09:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6705
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-12-18:09:51
INFO:tensorflow:Saving dic

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6795
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6795 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:loss = 0.17465104, step = 6796
INFO:tensorflow:Saving checkpoints for 6810 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:Loss for final step: 0.12873182.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-12-18:11:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6810
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-12-18:11:42
INFO:tensorflow:Saving dic

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6900
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6900 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:loss = 0.15578325, step = 6901
INFO:tensorflow:Saving checkpoints for 6915 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:Loss for final step: 0.14512013.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-12-18:13:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-6915
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-12-18:13:36
INFO:tensorflow:Saving dic

...

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-8790
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 8790 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:loss = 0.14516068, step = 8791
INFO:tensorflow:Saving checkpoints for 8805 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:Loss for final step: 0.13069436.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-12-18:47:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-8805
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-12-18:47:22
INFO:tensorflow:Saving dic

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-8895
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 8895 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:loss = 0.12901111, step = 8896
INFO:tensorflow:Saving checkpoints for 8910 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:Loss for final step: 0.11662796.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-12-18:49:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-8910
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-12-18:49:14
INFO:tensorflow:Saving dic

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 9000 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:loss = 0.1419371, step = 9001
INFO:tensorflow:Saving checkpoints for 9015 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:Loss for final step: 0.13582467.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-12-18:51:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-9015
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-12-18:51:14
INFO:tensorflow:Saving dict

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-9105
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 9105 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:loss = 0.12316533, step = 9106
INFO:tensorflow:Saving checkpoints for 9120 into ./DNN_CUSTOM_REGRESSOR\model.ckpt.
INFO:tensorflow:Loss for final step: 0.14877138.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-12-18:53:05
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-9120
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-12-18:53:07
INFO:tensorflow:Saving dic

KeyboardInterrupt: 

In [14]:
generator = estimator.predict(input_fn=pred_input_fn(infer_features))
predictions = [next(generator) for i in range(len(infer_features))]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CUSTOM_REGRESSOR\model.ckpt-9150
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [15]:
ids = pd.read_csv('../input/test.csv.gz', compression='infer')['Id']

submission = pd.DataFrame()
submission['SalePrice'] = predictions
submission['Id'] = ids
submission.set_index('Id', inplace=True)

submission.to_csv('../output/12.dnn_custom_regressor[128,64]_500_epochs_dropout.csv')